<a href="https://colab.research.google.com/github/MateoProjects/TFM/blob/main/Stable_Diffusion_TFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Requirements

In [ ]:
!pip install diffusers==0.2.3 transformers scipy ftfy
!pip install "ipywidgets>=7,<8"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Login

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Download model

In [ ]:
from torch import autocast
from diffusers import StableDiffusionPipeline, LMSDiscreteScheduler
import torch
# this will substitute the default PNDM scheduler for K-LMS  
lms = LMSDiscreteScheduler(
    beta_start=0.00085, 
    beta_end=0.012, 
    beta_schedule="scaled_linear"
)

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True).to("cuda")







In [ ]:
#StableDiffusionPipeline.save_pretrained(pipe, save_directory=".")
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

## Prompt images

In [ ]:
#@title Inputs { run: "auto", display-mode: "form" }
imagine = 'A samurai' #@param {type:"string"}
style = 'Sinister' #@param ["Oil painting", "Cosmic", "Sinister", "Anime", "Manga", "Fantasy"]
other = 'futuristic' #@param {type:"string"}
num_inference_steps = 50 #@param {type:"integer"}
num_images = 3 #@param {type:"integer"}
height = 512 #@param [256, 512, 1024]
width = 512 #@param [256, 512, 1024]
rows = 1 #@param {type:"integer"}
cols = 3 #@param {type:"integer"}

imagine = "% s % s % s"%(imagine, style, other)
num_inference_steps = num_inference_steps if num_inference_steps > 0 else 50



A samurai Sinister futuristic


In [ ]:
#@title Push play for imagine  {display-mode: "form"}
prompt = [imagine] * num_images

with autocast("cuda"):
  images = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps)["sample"]

grid = image_grid(images, rows=rows, cols=cols)
grid
#images[0]

['A samurai Sinister futuristic', 'A samurai Sinister futuristic', 'A samurai Sinister futuristic']


0it [00:00, ?it/s]